# Web Server logs Analysis

De forma general, un server log es un archivo de log generado por el servidor con una
lista de las actividades que se ejecutan. En este caso tenemos un web server log el cuál
mantiene un historial de las peticiones realizadas a la página. Este tipo de server logs
tienen un formato standard (Common Log Format). Y es una práctica general, el
analizar estos logs para sacar distintas conclusiones, localizar ataques, errores
comunes, etc.
En nuestro caso tenemos el dataset de los web server logs de la NASA. Qué están
compuestos por este tipo de registros:

133.43.96.45 - - [01/Aug/1995:00:00:23 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713

Por lo que tenemos los siguientes campos:

1. **Host**: 133.43.96.45
2. **User-identifier**: en este dataset, todos estos campos estarán con un “-“ que significa que faltan esos datos, por lo que obviaremos este campo.
3. **Userid**: al igual que el anterior campo, también será obviado.
4. **Date**: 01/Aug/1995:00:00:23 -0400, como podemos ver está en formato dd/MMM/yyyy:HH:mm:ss y el campo final “-0400” sería el timezone que en este caso omitiremos, además haremos una transformación de los meses a forma numérica.
5. **Request Method**: GET, existen distintos métodos de petición aquí puedes obtener más información: link
6. **Resource**: /images/launch-logo.gif, sería el recurso al que se accede en esta petición.
7. **Protocol**: HTTP/1.0, y por ultimo en esta parte entre comillas tendríamos el protocolo utilizado al ser logs de 1995, seguramente sea el único protocolo utilizado.
8. **HTTP status code**: 200, existen distintos códigos de estado de HTTP en el link a continuación tienes más información: link
9. **Size**: 1713, y como ultimo campo tendríamos el tamaño del objeto recibido por el cliente en bytes. En casos de error del cliente, este campo no se encuentra por lo que al igual que en los userid, será indicado con un “-“, tenerlo en cuenta.

Ahora que ya entendemos que se encuentra dentro de nuestro web server log, vamos a pasar
a analizarlo. Primero debemos cargar el archivo como un archivo de texto normal y realizar las
transformaciones pertinentes, a la hora de limpiar y estructurar nuestro dataset utilizaremos
expresiones regulares para recoger los campos que necesitamos.
Guardaremos nuestro nuevo DataFrame ya estructurado en formato parquet. Y de este
leeremos para realizar nuestro análisis.

Consultas a realizar:
- ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.
- ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.
- ¿Y los métodos de petición (verbos) más utilizados?
- ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?
- Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log.
- ¿Qué días la web recibió más tráfico?
- ¿Cuáles son los hosts son los más frecuentes?
- ¿A qué horas se produce el mayor número de tráfico en la web?
- ¿Cuál es el número de errores 404 que ha habido cada día?


### Inicio de la SparkSession

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Nasa")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2111011.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1643622693861)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3c08904f


### Carga del csv

In [4]:
// Cuando ponemos el asterisco al final de access_log conseguimos cargar los dos documentos (Jul y Aug) sin hacer un union

val directorio = "C:/Users/sara.arribas/Documents/Big Data/NASA/access_log_*"

val nasaDF = spark.read.format("text")
                            .load(directorio)

directorio: String = C:/Users/sara.arribas/Documents/Big Data/NASA/access_log_*
nasaDF: org.apache.spark.sql.DataFrame = [value: string]


In [17]:


nasaDF.show(5, false)
print(nasaDF.count())

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


In [ ]:
Vamos a borrar todas las filas que no tengan Protocolo

### Aplicando Regex para limpiar el formato

In [203]:
import org.apache.spark.sql.functions._
val nasa_pattern = """^(.*) - - \[(\d\d\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})\] "(\S+) (\S+) *(\S*)" (\d*) (\d+|-|\s)$"""

val nasa_parsedDF = nasaDF.select(regexp_extract($"value", exp = nasa_pattern, groupIdx = 1).alias("Host"),
                                  regexp_extract($"value", exp = nasa_pattern, groupIdx = 2).alias("Date"),
                                  regexp_extract($"value", exp = nasa_pattern, groupIdx = 3).alias("Request_Method"),
                                  regexp_extract($"value", exp = nasa_pattern, groupIdx = 4).alias("Resource"),
                                  regexp_extract($"value", exp = nasa_pattern, groupIdx = 5).alias("Protocol"),
                                  regexp_extract($"value", exp = nasa_pattern, groupIdx = 6).alias("HTTP_status_code"),
                                  regexp_extract($"value", exp = nasa_pattern, groupIdx = 7).alias("Size"))
//val nasa_noNulls = nasa_parsedDF.na.fill(0,"Size")
//nasa_parsedDF.show(10, false)
print(nasa_parsedDF.count())



3461613

import org.apache.spark.sql.functions._
nasa_pattern: String = ^(.*) - - \[(\d\d\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})\] "(\S+) (\S+) *(\S*)" (\d*) (\d+|-|\s)$
nasa_parsedDF: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 5 more fields]


In [204]:
val nasa_SizeDF = nasa_parsedDF.withColumn("Size", when($"Size"===""||$"Size"==="-",0).otherwise($"Size"))

nasa_SizeDF: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 5 more fields]


In [169]:
nasa_SizeDF.where($"Protocol"==="").count()

res146: Long = 6601


### Guardo el archivo en parquet

In [205]:
val parquetPath = "/tmp/data/parquet/parquet_nasa"
nasa_parsedDF.write.format("parquet").save(parquetPath)

parquetPath: String = /tmp/data/parquet/parquet_nasa


### ¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.

Borramos las filas con protocolo vacío

In [170]:
nasa_SizeDF.groupBy($"Protocol").count().show(false)

+-------------+-------+
|Protocol     |count  |
+-------------+-------+
|HTTP/*       |13     |
|a            |1      |
|             |6601   |
|HTTP/V1.0    |279    |
|HTTP/1.0     |3454715|
|STS-69</a><p>|4      |
+-------------+-------+



In [171]:
nasa_SizeDF.select("*").where($"Protocol"==="STS-69</a><p>").show(false)

+--------------+--------------------------+--------------+---------------------------------------------+-------------+----------------+----+
|Host          |Date                      |Request Method|Resource                                     |Protocol     |HTTP status code|Size|
+--------------+--------------------------+--------------+---------------------------------------------+-------------+----------------+----+
|hertz.njit.edu|10/Jul/1995:23:34:41 -0400|GET           |/shuttle/missions/sts-69/mission-sts-69.html>|STS-69</a><p>|404             |0   |
|139.169.52.155|12/Jul/1995:14:32:30 -0400|GET           |/shuttle/missions/sts-69/mission-sts-69.html>|STS-69</a><p>|404             |0   |
|auriga.ta3.sk |27/Jul/1995:04:50:09 -0400|GET           |/shuttle/missions/sts-69/mission-sts-69.html>|STS-69</a><p>|404             |0   |
|auriga.ta3.sk |27/Jul/1995:04:50:11 -0400|GET           |/shuttle/missions/sts-69/mission-sts-69.html>|STS-69</a><p>|404             |0   |
+------------

### ¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.

In [180]:
nasa_SizeDF.groupBy($"HTTP status code").count().orderBy($"count").show(false)

+----------------+-------+
|HTTP status code|count  |
+----------------+-------+
|501             |41     |
|500             |65     |
|403             |225    |
|                |1760   |
|404             |20802  |
|302             |73006  |
|304             |266764 |
|200             |3098950|
+----------------+-------+



### Y los métodos de petición (verbos) más utilizados?

In [179]:
nasa_SizeDF.groupBy($"Request Method").count().orderBy($"count").show(false)

+--------------+-------+
|Request Method|count  |
+--------------+-------+
|POST          |222    |
|              |1760   |
|HEAD          |7915   |
|GET           |3451716|
+--------------+-------+



### ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?


In [196]:
import org.apache.spark.sql.functions._
nasa_SizeDF.groupBy($"Resource").agg(sum($"Size").alias("Size")).orderBy(desc("Size")).show(5,false)

+------------------------------------------------------------+-------------+
|Resource                                                    |Size         |
+------------------------------------------------------------+-------------+
|/shuttle/missions/sts-71/movies/sts-71-launch.mpg           |3.195286412E9|
|/shuttle/missions/sts-71/movies/sts-71-mir-dock.mpg         |1.409035595E9|
|/shuttle/missions/sts-71/movies/sts-71-tcdt-crew-walkout.mpg|1.137114616E9|
|/shuttle/missions/sts-70/movies/sts-70-launch.mpg           |1.098853893E9|
|/shuttle/technology/sts-newsref/stsref-toc.html             |1.061238918E9|
+------------------------------------------------------------+-------------+
only showing top 5 rows



import org.apache.spark.sql.functions._


### Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log.

In [197]:
nasa_SizeDF.groupBy($"Resource").count().orderBy(desc("count")).show(5,false)

+----------------------------+------+
|Resource                    |count |
+----------------------------+------+
|/images/NASA-logosmall.gif  |208714|
|/images/KSC-logosmall.gif   |164970|
|/images/MOSAIC-logosmall.gif|127908|
|/images/USA-logosmall.gif   |127074|
|/images/WORLD-logosmall.gif |125925|
+----------------------------+------+
only showing top 5 rows



### ¿Qué días la web recibió más tráfico?


In [223]:
val nasa_DiasDF = nasa_SizeDF.withColumn("Dias",substring(col("date"), 1, 11))
nasa_DiasDF.groupBy("Dias").agg(count("Dias").alias("count")).orderBy(desc("count")).show(5)

+-----------+------+
|       Dias| count|
+-----------+------+
|13/Jul/1995|133952|
|06/Jul/1995|100956|
|05/Jul/1995| 94572|
|12/Jul/1995| 92230|
|31/Aug/1995| 89825|
+-----------+------+
only showing top 5 rows



nasa_DiasDF: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 6 more fields]


### ¿Cuáles son los hosts son los más frecuentes?


In [215]:
nasa_SizeDF.groupBy($"Host").count().orderBy(desc("count")).show(5,false)

+--------------------+-----+
|Host                |count|
+--------------------+-----+
|piweba3y.prodigy.com|21988|
|piweba4y.prodigy.com|16437|
|piweba1y.prodigy.com|12825|
|edams.ksc.nasa.gov  |11964|
|163.206.89.4        |9697 |
+--------------------+-----+
only showing top 5 rows



### ¿A qué horas se produce el mayor número de tráfico en la web?


In [222]:
val nasa_HorasDF = nasa_SizeDF.withColumn("Hora",substring(col("Date"), 13, 2))
nasa_HorasDF.groupBy("Hora").agg(count("Hora").alias("count")).orderBy(desc("count")).show(24)

+----+------+
|Hora| count|
+----+------+
|  15|230585|
|  12|227207|
|  13|225294|
|  14|223771|
|  16|217488|
|  11|211000|
|  10|193699|
|  09|178587|
|  17|178366|
|  08|149121|
|  18|146002|
|  22|131334|
|  19|131013|
|  21|129826|
|  20|129658|
|  23|123845|
|  00|110205|
|  07|101330|
|  01| 91542|
|  02| 77789|
|  03| 67346|
|  06| 66480|
|  05| 59441|
|  04| 58924|
+----+------+
only showing top 24 rows



nasa_HorasDF: org.apache.spark.sql.DataFrame = [Host: string, Date: string ... 6 more fields]


### ¿Cuál es el número de errores 404 que ha habido cada día?

In [224]:
nasa_DiasDF.where($"HTTP_status_code"==="404").groupBy($"Dias").count().orderBy(desc("count")).show(5,false)

+-----------+-----+
|Dias       |count|
+-----------+-----+
|19/Jul/1995|639  |
|06/Jul/1995|636  |
|30/Aug/1995|567  |
|07/Jul/1995|565  |
|07/Aug/1995|532  |
+-----------+-----+
only showing top 5 rows

